In [1]:
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from pyiomica_master import pyiomica as pio
from pyiomica_master.pyiomica import visualizationFunctions
from pyiomica_master.pyiomica import visibilityGraphCommunityDetection
import matplotlib as mpl
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tqdm import tqdm
%matplotlib inline

Loading PyIOmica 1.3.3 (https://github.com/gmiaslab/pyiomica by G. Mias Lab)


/data/user/home/sahaj432/Desktop/code_organized/G_signal_feature_reduction_Project/New_neural_code_SNR_1.25_v/pyiomica_master/pyiomica/visibilityGraphAuxiliaryFunctions.py:91: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(cache=True)


In [2]:
x_frame1_05 = np.genfromtxt('/home/sahaj432/Desktop/dataset/neural_4_class/x_frame1_1.25_seed.csv', delimiter=',')
x_frame2_05 = np.genfromtxt('/home/sahaj432/Desktop/dataset/neural_4_class/x_frame2_1.25_seed.csv', delimiter=',')
x_frame3_05 = np.genfromtxt('/home/sahaj432/Desktop/dataset/neural_4_class/x_frame3_1.25_seed.csv', delimiter=',')
x_framen_05= np.genfromtxt('/home/sahaj432/Desktop/dataset/neural_4_class/x_frame_noise_1.25_seed.csv', delimiter=',')

In [3]:
x=np.concatenate((x_frame1_05,x_frame2_05,x_frame3_05,x_framen_05[:6000]),axis=0)
y=np.concatenate((np.zeros(len(x_frame1_05)),np.zeros(len(x_frame2_05))+1,np.zeros(len(x_frame3_05))+2,np.zeros(len(x_framen_05[:6000]))+3),axis=0)

In [4]:
X, Y = shuffle(x,y, random_state=1)

In [5]:
def calculate_adj(data):
    temp1=[]
    for p, i in enumerate(data):
        #print(f'Converting ==> {p+1} \r', end="",flush=True)
        vs_out = visibilityGraphCommunityDetection.createVisibilityGraph(i, range(len(i)), "natural", weight='tan')
        vs_out=np.asarray(vs_out)
        temp1.append(vs_out)
        
    return temp1

In [6]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)

Number of CPU cores: 48


In [7]:
df=np.array_split(X, num_cores)
print(len(df[0]))

533


In [8]:
with Pool(processes=num_cores) as pool:
    results = pool.map(calculate_adj, df)

pool.close()
pool.join()

In [9]:
x_adj=[]
for submatrix in results:
    x_adj.extend(submatrix)

In [10]:
print(len(x_adj))

25578


In [11]:
x_adj=np.array(x_adj)

In [12]:
def graph_index_complexity(matrix):
    N = len(matrix) # Number of nodes
    E = np.sum(matrix > 0) / 2 # Number of edges (since matrix is symmetric for undirected graphs)
    
    if N == 0 or E == 0:
        return 0
    
    GIC = (E / N) * np.log(N)
    return GIC

In [13]:
def degree(data): #The node strengths (or weighted degrees) 
    return np.sum(data, axis=0).flatten()

In [14]:
import networkx as nx
def runall(data):
    temp1=[]
    for i in data:
        G = nx.from_numpy_array(i, parallel_edges=False)
        
        cpl=[nx.average_shortest_path_length(G)]
        ge=[nx.global_efficiency(G)]
        le=[nx.local_efficiency(G)]
        ac=[nx.degree_assortativity_coefficient(G)]
        d=degree(i).tolist()
        bc=list(nx.betweenness_centrality(G).values())
        cc=list(nx.clustering(G).values())
        temp1.append(cpl+ge+le+ac+d+bc+cc) #+cpl+ge+le+ac+cc
    return temp1

In [15]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)


df1=np.array_split(x_adj[:], num_cores)


print(len(df[0]))




Number of CPU cores: 48
533


In [16]:

import time
start_time = time.time()
with Pool(processes=num_cores) as pool1:
    results1 = pool1.map(runall, df1)

pool1.close()
pool1.join()

print("--- %s seconds ---" % (time.time() - start_time))


--- 30.50870156288147 seconds ---


In [17]:
x_g=[]
for submatrix in results1:
    x_g.extend(submatrix)

In [18]:
x_g=np.array(x_g)


In [19]:
print(x_g.shape)

(25578, 172)


In [20]:
x_train, x_temp, y_train, y_temp = train_test_split(x_g, Y, test_size=0.30, random_state=42)
x_val,x_test,y_val,y_test=train_test_split(x_temp,y_temp,test_size=0.50, random_state=42)

In [21]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(17904, 172)
(3837, 172)
(3837, 172)


In [22]:
import tensorflow as tf


2023-08-13 09:35:30.231265: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-08-13 09:35:30.231296: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [23]:
input_shape = (x_train.shape[1],1)
num_classes = 4
model3 = tf.keras.models.Sequential()
model3.add(tf.keras.layers.Conv1D(filters= 32, kernel_size=3, activation='relu',padding='same',input_shape= input_shape))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Flatten())

model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dropout(0.1))
model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

2023-08-13 09:35:55.893246: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-08-13 09:35:55.893277: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-13 09:35:55.893296: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0158): /proc/driver/nvidia/version does not exist
2023-08-13 09:35:55.893582: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [24]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 172, 32)           128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 86, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 86, 32)            3104      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 43, 32)           0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 1376)              0         
                                                                 
 dense (Dense)               (None, 100)               1

In [25]:
model3.compile(loss='SparseCategoricalCrossentropy', optimizer='Adam', metrics=['acc'])


In [26]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model



checkpoint = ModelCheckpoint('F2.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


In [27]:
history_log = model3.fit(x_train, y_train, epochs=30, verbose=1, validation_data=(x_test, y_test), callbacks=[checkpoint])


Epoch 1/30
560/560 [==============================] - 3s 4ms/step - loss: 0.2069 - acc: 0.9255 - val_loss: 0.0775 - val_acc: 0.9721
Epoch 2/30
560/560 [==============================] - 2s 4ms/step - loss: 0.0887 - acc: 0.9714 - val_loss: 0.0593 - val_acc: 0.9784
Epoch 3/30
560/560 [==============================] - 2s 4ms/step - loss: 0.0775 - acc: 0.9725 - val_loss: 0.0681 - val_acc: 0.9758
Epoch 4/30
560/560 [==============================] - 2s 4ms/step - loss: 0.0675 - acc: 0.9775 - val_loss: 0.0776 - val_acc: 0.9737
Epoch 5/30
560/560 [==============================] - 2s 4ms/step - loss: 0.0630 - acc: 0.9792 - val_loss: 0.0583 - val_acc: 0.9807
Epoch 6/30
560/560 [==============================] - 2s 4ms/step - loss: 0.0581 - acc: 0.9813 - val_loss: 0.0497 - val_acc: 0.9828
Epoch 7/30
560/560 [==============================] - 2s 4ms/step - loss: 0.0565 - acc: 0.9807 - val_loss: 0.0599 - val_acc: 0.9794
Epoch 8/30
560/560 [==============================] - 2s 4ms/step - loss: 0.

Epoch 29/30
560/560 [==============================] - 2s 4ms/step - loss: 0.0193 - acc: 0.9931 - val_loss: 0.0714 - val_acc: 0.9810
Epoch 30/30
560/560 [==============================] - 2s 4ms/step - loss: 0.0205 - acc: 0.9934 - val_loss: 0.0671 - val_acc: 0.9823


In [28]:
model3.save_weights('/home/sahaj432/Desktop/code_organized/G_signal_feature_reduction_Project/New_neural_code_SNR_1.25_v/F2.h5')


In [29]:
model3.load_weights('/home/sahaj432/Desktop/code_organized/G_signal_feature_reduction_Project/New_neural_code_SNR_1.25_v/F2.h5')


In [30]:
predictions = model3.predict(x_test)
pred_arg = np.argmax(predictions, axis=1)

In [31]:
from sklearn.metrics import confusion_matrix
cnf_matrix=confusion_matrix(y_test,pred_arg)
print("cnf_matrix_2.0_70")
print("")
print(cnf_matrix)

cnf_matrix_2.0_70

[[960   1   0   2]
 [  0 965  34   2]
 [  0  19 963   0]
 [  8   1   1 881]]


In [32]:
from sklearn.metrics import accuracy_score, recall_score,precision_score,roc_auc_score,roc_curve,f1_score


In [33]:
print(accuracy_score(y_test,pred_arg)*100)
print(f1_score(y_test,pred_arg,average='macro')*100)
print(recall_score(y_test,pred_arg, average="weighted")*100)
print(precision_score(y_test,pred_arg, average="weighted")*100)

98.22778212144905
98.26153493865466
98.22778212144905
98.23445017716544
